# Introduction to simple-C-elegans

*original available online: https://github.com/adammarblestone/simple-C-elegans/blob/master/SimulateConnectome.py*

This is essentially a single python script that simulates the nervous system of the C. elegans; with many caveats known to the authors.  The goal is purposeful simplicity to be able to demonstrate the concepts surrounding simulating a nervous system with a reasonable amount of biological detail.  This walkthrough will explain the simple-C-elegans code and how it works.  

A more robust approach is being taken in the OpenWorm project that involves many pieces of infrastructure that are more sophisticated, but also less simple.  Therefore, in addition to explaining how this code works, the larger OpenWorm infrastructure that is responsible for various steps will also be explained along the way.

Comments at the top say:
    
>Simulates the dynamics of part or all of the C. elegans neural network.
>This code is *deliberately simple* in order to help the user gain intuition for the qualitative properties of 
the network dynamics.
>Key references include:
> 1) Wicks, Stephen R., Chris J. Roehrig, and Catharine H. Rankin. "A dynamic network simulation of the nematode tap withdrawal circuit: predictions concerning synaptic function using behavioral criteria." The Journal of neuroscience 16.12 (1996): 4017-4031.
> 2) Kunert, James, Eli Shlizerman, and J. Nathan Kutz. "Low-dimensional functionality of complex network dynamics: Neurosensory integration in the Caenorhabditis elegans connectome." Physical Review E 89.5 (2014): 052805.
>3) http://www.openworm.org/ and the associated resources

First, some practicalities.  The [sci-py libraries](https://www.scipy.org/install.html) are needed to make this run.  One particularly nice setup can be found here: https://github.com/jupyter/docker-stacks/tree/master/scipy-notebook with the setup via Docker.

In addition, installing neuroml will be helpful:

In [9]:
!sudo apt-get update
!sudo apt-get install -y libxml2-dev libxslt-dev
!pip2 install libneuroml

Hit http://security.debian.org jessie/updates InRelease
Get:1 http://security.debian.org jessie/updates/main amd64 Packages [384 kB]
Ign http://httpredir.debian.org jessie InRelease
Hit http://httpredir.debian.org jessie-updates InRelease
Hit http://httpredir.debian.org jessie Release.gpg
Get:2 http://httpredir.debian.org jessie-updates/main amd64 Packages [17.6 kB]
Hit http://httpredir.debian.org jessie Release
Get:3 http://httpredir.debian.org jessie/main amd64 Packages [9,032 kB]
Fetched 9,433 kB in 7s (1,197 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libxslt1-dev' instead of 'libxslt-dev'
libxml2-dev is already the newest version.
libxslt1-dev is already the newest version.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
  Using cached libNeuroML-0.2.18.tar.gz
  Using cached lxml-3.6.1.tar.gz
  Running setup.py bdist_wheel for libneuroml ... - \ done
  Stored

After these installs, now the imports from the script should work:

In [10]:
import neuroml
import neuroml.loaders as loaders
import neuroml.writers as writers
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import csv
from scipy.integrate import odeint
import numpy as np
import random
import matplotlib.animation as animation
import time
import pickle
import os

/opt/conda/envs/python2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
